### <b>문제 4-1 : OpenAI에서 Ollama Qwen3로 RAG 시스템 변경하기</b>
<b>문제 설명</b><br>
: 주어진 OpenAI 기반 RAG 시스템 코드를 로컬 Ollama Qwen3 모델을 사용하도록 변경하는 연습문제입니다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('./data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기 (한국어 최적화)
    chunk_overlap=200,      # 중복 부분 (맥락 보존)
    separators=["\n\n", "\n", ".", " ", ""] # 자연스러운 분할을 위한 구분자
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("==> 3. 벡터화 → 임베딩으로 변환")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 고성능 임베딩 모델
    dimensions=1536
)

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # 상위 6개 관련 문서 검색
)
print(" Retriever 설정 완료")

print("===> 6. 생성 → LLM으로 답변 생성")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1,
    max_tokens=1500
)

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{retrieveDocuments}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")

# ===================================
# 7. QA 체인 생성
# ===================================
print("\n ===> 7.  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt, "document_variable_name": "retrieveDocuments"},
    return_source_documents=True
)
print("  RAG 파이프라인 구축 완료!")

# ===================================
# 8. 테스트 질문들
# ===================================
test_questions = [
    "BlueJ에서 객체를 생성하는 방법은 무엇인가요?",
    "컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?", 
    "디버깅을 위해 중단점을 설정하는 방법을 알려주세요",
    "코드패드는 무엇이고 어떻게 사용하나요?",
    "애플릿을 만들고 실행하는 방법을 설명해주세요"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 9. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)
